In [1]:
import pandas as pd
import numpy as np

In [2]:
#importing the datasets
movies_metadata = pd.read_csv("raw_datasets/movies_metadata.csv")
ratings = pd.read_csv("raw_datasets/ratings.csv")

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### I got a DtypeWarning while loading the datasets which means that some values' dtype does not match the dtype of the column. Before commiting any changes to a dataset I will create copies of them.

In [3]:
mmd = movies_metadata.copy()
rts = ratings.copy()

## movies_metadata set  data preperation

In [4]:
#checking general information of movies_metadata
mmd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
#dropping non related columns
mmd.drop(mmd.columns[[0,1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23]], axis=1, inplace=True)

#checking for null values
mmd.isnull().sum()

genres    0
id        0
title     6
dtype: int64

In [6]:
#finding the position of the NaN values
mmd.loc[pd.isna(mmd['title']), :]

genres          id title
19729  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...       82663   NaN
19730  [{'name': 'Carousel Productions', 'id': 11176}...  1997-08-20   NaN
29502  [{'id': 16, 'name': 'Animation'}, {'id': 878, ...      122662   NaN
29503  [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...  2012-09-29   NaN
35586  [{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...      249260   NaN
35587  [{'name': 'Odyssey Media', 'id': 17161}, {'nam...  2014-01-01   NaN

**I assumed that missing titles are movies which were removed from the platform so I decided to remove them. However, before removing I need to check if any users had voted on them**

In [7]:
#checking the ids of the missing movies in the ratings dataset 
rts.loc[((rts['movieId'] == 82663) | (rts['movieId'] == 122662) | (rts['movieId'] == 249260))]

Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []

In [8]:
#none of the ids were not present in the rating history so I can safely remove them
mmd.dropna(inplace=True)
mmd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  45460 non-null  object
 1   id      45460 non-null  object
 2   title   45460 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [9]:
#changing the columns to the correct dtype
mmd['title'] = mmd['title'].astype('string', copy=False)
mmd['id'] = mmd['id'].astype(np.int64, copy=False)
mmd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  45460 non-null  object
 1   id      45460 non-null  int64 
 2   title   45460 non-null  string
dtypes: int64(1), object(1), string(1)
memory usage: 1.4+ MB


**the genres column contains a list of dictionaries for each movie. Each dictionary contains the movie's genre name and the genre's id. I will use the Abstract Syntax Tree module in order to extract only the genre names and replace the list of dictionaries with the genre names**

In [10]:
import ast
genres = mmd.genres.apply(ast.literal_eval).apply(lambda x : [i['name'] for i in x])
mmd['genres'] = genres
mmd.head()

genres     id                        title
0   [Animation, Comedy, Family]    862                    Toy Story
1  [Adventure, Fantasy, Family]   8844                      Jumanji
2             [Romance, Comedy]  15602             Grumpier Old Men
3      [Comedy, Drama, Romance]  31357            Waiting to Exhale
4                      [Comedy]  11862  Father of the Bride Part II

In [11]:
#checking for duplicate values in title column
mmd['title'].value_counts()

Cinderella              11
Alice in Wonderland      9
Hamlet                   9
Les Misérables           8
Beauty and the Beast     8
                        ..
Killed by Lightning      1
Broadway Danny Rose      1
The Coalition            1
Scorcher                 1
Crazy Beautiful You      1
Name: title, Length: 42277, dtype: Int64

**There are some duplicate titles, I can assume that they are all the same movie. The only way to check if the duplicates are the same movie is by looking at their id**

In [12]:
mmd.loc[mmd['title'] == 'Cinderella']

genres      id       title
993          [Family, Fantasy, Animation, Romance]   11224  Cinderella
13076  [TV Movie, Family, Fantasy, Music, Romance]   42884  Cinderella
23507                             [Fantasy, Drama]   92349  Cinderella
23518                 [Animation, Family, Fantasy]  105875  Cinderella
28391                           [Family, TV Movie]  261985  Cinderella
28664            [Romance, Fantasy, Family, Drama]  150689  Cinderella
34254                    [Comedy, Family, Fantasy]   42651  Cinderella
35593                             [Drama, Romance]   44459  Cinderella
35595                                           []  289673  Cinderella
41074   [Fantasy, Horror, Science Fiction, Family]  114108  Cinderella
45406                            [Comedy, Romance]   98604  Cinderella

**From the data I can see that although the titles are the same, the ids and genres differ from each title. As a result, they are not the same movie**

In [13]:
#checking for duplicate id
mmd['id'].value_counts()

141971    3
22649     2
69234     2
109962    2
14788     2
         ..
376934    1
86751     1
19164     1
29403     1
276895    1
Name: id, Length: 45430, dtype: int64

**There are some duplicate ids. I am going to look if they are the same movie or different movies which accidently got the same id**

In [14]:
mmd.loc[mmd['id'] == 141971]

genres      id     title
13261  [Thriller, Mystery]  141971  Blackout
13375  [Thriller, Mystery]  141971  Blackout
16764  [Thriller, Mystery]  141971  Blackout

In [15]:
mmd.loc[mmd['id'] == 14788]

genres     id   title
10419  [Drama, Crime, Mystery]  14788  Bubble
12066  [Drama, Crime, Mystery]  14788  Bubble

**I can safely remove the duplicated id.**

In [16]:
mmd.drop_duplicates(subset=['id'], inplace=True)
mmd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45430 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  45430 non-null  object
 1   id      45430 non-null  int64 
 2   title   45430 non-null  string
dtypes: int64(1), object(1), string(1)
memory usage: 1.4+ MB


## ratings data preperation

In [17]:
rts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [18]:
rts.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [19]:
rts.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [20]:
mmd.to_csv('movies_md.csv', index=False)
rts.to_csv('ratings_nw.csv', index=False)